## 0.0 Libraries

In [2]:
import os
import sys
import copy
# For plotting and numerics.
import numpy as np
import seaborn as sns

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json
import matplotlib.pyplot as plt
#import sympy

# For LAS loading and data management.
import lasio
import pandas as pd
import glob
from scipy.signal import butter, lfilter, freqz
from scipy.stats import linregress
from obspy.io.segy.segy import _read_segy
import math

## 0.1 Variables

In [ ]:
#Sample rate of seismic data
sample_rate= 2

#Start time of seismic data
start_seis = 900

#End time of seismic data
end_seis = 1802

#Inline number
inline_num = 2334

#Shape parameter for Kaiser window
beta=50

#Normalized amplitude treshold, smaller amplitudes will be skipped during computation
treshold=0.2

#Number of samples of the operator
num=100

# Color of heatmap
colorscale = json.load(open('data/gap.json'))

## 1.0 Pasrsing seismic

In [ ]:
def parse_inline(inline,file_path):
    data = []
    cross = []
    from obspy.io.segy.segy import _read_segy
    stream = _read_segy(file_path,endian='>', headonly=True)
    for trace in stream.traces:
        if trace.header.for_3d_poststack_data_this_field_is_for_in_line_number == inline:
            cross.append(trace.header.for_3d_poststack_data_this_field_is_for_cross_line_number)
            trace_array = trace.unpack_data().tolist()
            data.append(trace_array)
    r_data = [data,cross]
    return r_data

parsed_segy = parse_inline(inline_num, 'data/Stack_izmenennyi.sgy')
panel_IP = np.transpose(parsed_segy[0])  
xbin = parsed_segy[1] 

panel_seis = np.transpose(parsed_segy[0])
time = np.arange(start_seis, end_seis, sample_rate)

## 1.0 Plot

In [ ]:
fig=go.Figure(data=go.Heatmap(z=panel_seis,x=xbin,y=time,
                              colorscale="Greys",  
                              colorbar=dict(
                               title="Ampl")))
fig.update_layout(title=go.layout.Title(text="Initial model"),
                xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='XBIN')),
                 yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Time[ms]')))
fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()



## 2.0 Parsing well

In [ ]:
import glob

Logs = pd.DataFrame()
temp = pd.DataFrame()

las_files = glob.glob('Well/P2_AI_correct.las')
for i, las in enumerate(las_files):
    well = lasio.read(las,encoding='utf-8',null_policy='common')
    logs_param = []
    temp = pd.DataFrame()
    for curve in well.curves:
        try:
            temp['ID'] = os.path.basename(las)[:5]
            temp[curve.mnemonic] = well[curve.mnemonic]
        except:
            pass
    if i == 0:   
        Logs = temp.copy()
    else:
        Logs = pd.concat([Logs,temp], axis=0)
        
grouped = Logs.groupby('ID')
AI_f021 = (grouped.get_group('P2_AI').AI).values
depth_f021 = (grouped.get_group('P2_AI').DEPTH).values

## 2.1 Interpolate Time-depth

In [ ]:
TD = np.loadtxt("TD/P2_Time_Depth.txt")
from scipy.interpolate import interp1d
f_td = interp1d(TD[:,0], TD[:,1], kind='linear',fill_value="extrapolate")
time_f021 = f_td(depth_f021)

## 2.2 Well spectrum

In [ ]:
n_log = AI_f021.shape[0]  # Number of samples.
k_log = np.arange(n_log-1)
Fs_log = 1 / (np.diff(time_f021/1000))  # Getting sampled frequencies.
T_log = n_log / Fs_log
freq_log = k_log / T_log
freq_log = freq_log[range(n_log//2)]  # One side frequency range.

spec_log = np.fft.fft(AI_f021) / n_log  # FFT computing and normalization.
spec_log = spec_log[range(n_log//2)]

#Calculate logarithmic well spectrum
freq_log = np.log(freq_log[1:1500])
spec_log = np.log(spec_log[1:1500])


## 2.3 Linear regression on logarithmic well spectrum

In [ ]:
slope, intercept, rvalue, pvalue, stderr = linregress(freq_log,np.abs(spec_log))
print ('Regression results:')
print ("Intercept:", intercept)
print ("Slope    :", slope)
print ("R-value  :", rvalue)
lintrend=intercept+slope*freq_log

## 2.0 + 2.3 Plot

In [ ]:
fig = make_subplots(rows=1, cols=2,
                   subplot_titles=("AI log", "Ampl spectrum of AI log"),column_widths=[0.3, 0.7])


fig.add_trace(go.Scatter(x=AI_f021,y=time_f021),row=1, col=1)
fig['layout']['yaxis']['autorange'] = "reversed"

fig.add_trace(go.Scatter(x=freq_log,y=np.abs(spec_log),mode='markers',marker=dict(color="blue", size=5)),row=1, col=2)

fig.add_trace(go.Scatter(x=freq_log,y=lintrend,line=dict(color="red")),row=1, col=2)

fig.update_xaxes(title_text="X BIN", row=1, col=1)
fig.update_yaxes(title_text="AI", row=1, col=1)
fig.update_xaxes(title_text="X BIN", row=1, col=2)
fig.update_yaxes(title_text="AI", row=1, col=2)

fig.update_layout(showlegend=False,height=600, width=800)



## 3.1  Seismic spectrum 

In [ ]:
#Расчет весового окна Хэмминга по теории
n_seis = len(time)
Hamming_window=[0]*n_seis
for i in range(n_seis):
    Hamming_window[i] = 0.53836 - 0.46164*np.cos(2*np.pi*i/(n_seis-1))
Hamming_window_teor = Hamming_window

In [ ]:
#Расчет АЧХ по всем трассам, а затем получение среднего АЧХ: FFT из НАМПАЯ + Расчет АКФ
n_seis = len(time)
half_n_seis = n_seis//2
min_time_op = 0
max_time_op = len(time)
xmin_op = 0
xmax_op = len(xbin)
FFT_seis = [0]*xmax_op
panel_seis_AKF = [0]*panel_seis
#Расчет АКФ для трасс - свертка трассы с собой
for i in range(xmax_op):
    panel_seis_AKF[:,i] = np.convolve(panel_seis[:,i], panel_seis[:,i], mode='same')

#C Хэммингом    
panel_seis_Hamming = [0]*panel_seis_AKF    
for i in range(xmax_op):    
    panel_seis_Hamming[:,i] = panel_seis_AKF[:,i]*Hamming_window
    FFT_seis[i] = np.abs(np.fft.fft(panel_seis_Hamming[:,i]))

fourier_spec = np.mean(FFT_seis, axis=0)

Re_spec = [0]*half_n_seis
Im_spec = [0]*half_n_seis
spec_seis = [0]*half_n_seis
for i in range(half_n_seis):
    Re_spec[i] = np.real(fourier_spec[i])
    Im_spec[i] = np.imag(fourier_spec[i])
    spec_seis[i] = (Re_spec[i]*Re_spec[i] + Im_spec[i]*Im_spec[i])**0.25

normseis = spec_seis/np.max(spec_seis)

fourier_freq = np.fft.fftfreq(n_seis, d = sample_rate/1000)
freq_seis = fourier_freq[:half_n_seis]

## 3.0 + 3.1 Plot

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(
    x=freq_seis,
    y=normseis,
    mode='lines',
    name='Normalized seismic AS'))
fig.update_layout(height=600, width=1000)

## 4.0 Operator CI

In [ ]:
#Calculate regression based trend well spectrum
WelltrendSpectrum = intercept*np.power(freq_seis,slope)

#Calculate residual spectrum
ResidualSpectrum=np.zeros(len(normseis))
for i in range(len(normseis)):
    if normseis[i]>treshold:
        ResidualSpectrum[i]= WelltrendSpectrum[i] / normseis[i]
        
#Normalize residual spectrum
ResidualSpectrum = ResidualSpectrum / np.max(ResidualSpectrum)

#Plot normalized seismic spectrum with well trend spectrum

thold=np.ones(len(freq_seis))
thold=treshold*thold

fig=go.Figure()
fig.add_trace(go.Scatter(
    x=freq_seis,
    y=normseis,
    mode='lines',
    name='Normalized seismic AS',
    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
    x=freq_seis,
    y=WelltrendSpectrum,
    mode='lines',
    name='Regression based AI AS',
    line=dict(color='orange')
))
fig.add_trace(go.Scatter(
    x=freq_seis,
    y=ResidualSpectrum,
    mode='lines',
    name='Frequency domain raw operator',
    line=dict(color='red')
))
fig.add_trace(go.Scatter(
    x=freq_seis,
    y=thold,
    mode='lines',
    name='Treshold',
    line=dict(color='grey')
))
fig.update_layout(title=go.layout.Title(text="Seismic spectrums"),
                 xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Frequency [Hz]'),range=[-1,125]),
                 yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Normalized Amplitude'),range=[-0.01,2]),
                height=600, width=1000)

In [ ]:
#Operator phase spectrum depending on polarity of the seismic data
#SEG normal(AI increase = through): phase=90; SEG reverse (AI increase = Peak): phase=-90
phase=-90

#Calculate dt
dt=1/(2*np.max(freq_seis))
df=freq_seis[2]-freq_seis[1]

#Setup complex amplitude spectrum for ifft with phase assumption
phase=np.radians(phase)
cspectrum_poz=ResidualSpectrum*(np.cos(phase)+1j*np.sin(phase))
cspectrum_neg=ResidualSpectrum*(np.cos(-1*phase)+1j*np.sin(-1*phase))
rev_cspectrum_neg=np.fliplr([cspectrum_neg])[0]
input_cspectrum=np.append(cspectrum_poz,rev_cspectrum_neg)

#Calculate ifft and reorder arrays
t_op=np.fft.ifft(input_cspectrum)
start_t=(-1/2)*dt*(len(input_cspectrum))
t_shift=np.linspace(start_t,-1*start_t,len(t_op))
t_op_shift=np.fft.ifftshift(t_op)

#Tapering
window=np.kaiser(len(t_shift),beta)
t_op_final=t_op_shift*window

#Operator trimming indexes
start_i=(int(len(t_shift)/2))-int(num/2)
stop_i=(int(len(t_shift)/2))+int(num/2)

## 4.0 Plot

In [ ]:

#Plot final time domain operator
fig=go.Figure()
fig.add_trace(go.Scatter(
    x=t_shift,
    y=np.real(t_op_final),
    mode='lines',
    name='Time domain operator'
))
fig.update_layout(title=go.layout.Title(text="Colour inversion operator"),
                 xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Time [s]')),
                 yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Amplitude')))
fig.show()

## 4.1 QC operator

In [ ]:

bt_op=np.fft.fft(t_op_final)
backfreq=np.fft.fftfreq(len(t_op_final),dt)

fig=go.Figure()
fig.add_trace(go.Scatter(
    x=backfreq,
    y=abs(bt_op),
    mode='lines',
    name='Tapered backtransformed operator',
    line=dict(color='red')
    
))
fig.add_trace(go.Scatter(
    x=freq_seis,
    y=ResidualSpectrum,
    mode='lines',
    name='Frequency domain raw operator',
    line=dict(color='grey')
))
fig.update_layout(title=go.layout.Title(text="QC tapering"),
                 xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Frequency (Hz)'),range=[0, 145]),
                 yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Normalized amplitude'),range=[-0.1, 1.1]))


In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(
    x=freq_seis,
    y=normseis,
    mode='lines',
    name='Normalized seismic AS',
    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
    x=freq_seis,
    y=WelltrendSpectrum,
    mode='lines',
    name='Regression based AI AS',
    line=dict(color='orange')
))
fig.add_trace(go.Scatter(
    x=backfreq,
    y=np.abs(bt_op),
    mode='lines',
    name='Tapered backtransformed operator',
    line=dict(color='red')
))
fig.add_trace(go.Scatter(
    x=freq_seis,
    y=thold,
    mode='lines',
    name='Treshold',
    line=dict(color='grey')
))
fig.update_layout(title=go.layout.Title(text="Seismic spectrums"),
                 xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Frequency [Hz]'),range=[-1,125]),
                 yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Normalized Amplitude'),range=[-0.01,2]),
                height=600, width=1000)

## 5.0 Свёртка с сейсмикой

In [ ]:
def convolve(t):
    return np.convolve(t, np.abs(t_op_final), mode='same')
ci = (np.apply_along_axis(convolve, axis=0, arr=panel_seis))

## 5.0 Plot

In [ ]:
fig=go.Figure(data=go.Heatmap(z=ci,x=xbin,y=time,
                                  colorscale= colorscale,
                                  colorbar=dict(
                                  tick0=0,
                                  dtick=0.001)
    ))
    

fig.update_layout(xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='XBIN')),
                 yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Time[ms]')),height=800, width=1000)

fig['layout']['yaxis']['autorange'] = "reversed"

fig.show()